# سیمانٹک کرنل اوپن بی این بی MCP سرور انٹیگریشن کے ساتھ

یہ نوٹ بک دکھاتی ہے کہ سيمانٹک کرنل کو اصل OpenBnB MCP سرور کے ساتھ کیسے استعمال کیا جائے تاکہ MCPStdioPlugin کا استعمال کرتے ہوئے اصلی Airbnb رہائش تلاش کی جا سکے۔ LLM تک رسائی کے لیے، یہ Microsoft Foundry استعمال کرتا ہے۔ اپنے ماحول کے متغیرات سیٹ کرنے کے لیے، آپ [Setup Lesson](/00-course-setup/README.md) کی پیروی کر سکتے ہیں۔


## درکار پیکجز کو امپورٹ کریں


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP پلگ ان کنکشن بنانا

ہم MCPStdioPlugin کا استعمال کرتے ہوئے [OpenBnB MCP سرور](https://github.com/openbnb-org/mcp-server-airbnb) سے کنیکٹ کریں گے۔ یہ سرور @openbnb/mcp-server-airbnb پیکیج کے ذریعے Airbnb کی تلاش کی سہولت فراہم کرتا ہے۔


## کلائنٹ بنانا

اس نمونے میں، ہم اپنے LLM تک رسائی کے لیے Microsoft Foundry استعمال کریں گے۔ یقین دہانی کر لیں کہ آپ کے ماحول کے متغیرات درست طریقے سے سیٹ ہیں۔


## ماحولیاتی ترتیب

Azure OpenAI کی ترتیبات کو ترتیب دیں۔ یقینی بنائیں کہ آپ نے درج ذیل ماحولیاتی متغیرات مقرر کیے ہیں:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP انضمام کو سمجھنا

یہ نوٹ بک **اصل OpenBnB MCP سرور** سے جڑتی ہے جو حقیقی Airbnb تلاش کی فعالیت فراہم کرتا ہے۔

### یہ کیسے کام کرتا ہے:

1. **MCPStdioPlugin**: MCP سرور کے ساتھ معیاری انپٹ/آؤٹ پٹ مواصلات استعمال کرتا ہے
2. **اصل NPM پیکج**: npx کے ذریعے `@openbnb/mcp-server-airbnb` ڈاؤن لوڈ اور چلاتا ہے
3. **لائیو ڈیٹا**: ان کے APIs سے اصل Airbnb پراپرٹی ڈیٹا واپس کرتا ہے
4. **فنگشن کی دریافت**: ایجنٹ خودکار طور پر MCP سرور سے دستیاب فنکشنز کو دریافت کرتا ہے

### دستیاب فنکشنز:

OpenBnB MCP سرور عموماً مہیا کرتا ہے:  
- **search_listings** - مقام اور معیار کے مطابق Airbnb پراپرٹیز تلاش کریں  
- **get_listing_details** - مخصوص پراپرٹیز کی تفصیلی معلومات حاصل کریں  
- **check_availability** - مخصوص تاریخوں کے لیے دستیابی چیک کریں  
- **get_reviews** - پراپرٹیز کے جائزے حاصل کریں  
- **get_host_info** - پراپرٹی میزبانوں کی معلومات حاصل کریں  

### شرائط:

- آپ کے سسٹم پر **Node.js** انسٹال ہو  
- MCP سرور پیکج ڈاؤن لوڈ کرنے کے لیے **انٹرنیٹ کنکشن**  
- **NPX** دستیاب ہو (Node.js کے ساتھ آتا ہے)  

### کنکشن کی جانچ:

آپ MCP سرور کو دستی طور پر اس کمانڈ کو چلانے سے جانچ سکتے ہیں:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
یہ OpenBnB MCP سرور کو ڈاؤن لوڈ اور شروع کرے گا، جس سے Semantic Kernel پھر حقیقی Airbnb ڈیٹا کے لیے جڑتا ہے۔


## OpenBnB MCP سرور کے ساتھ ایجنٹ چلانا

اب ہم AI ایجنٹ چلائیں گے جو OpenBnB MCP سرور سے منسلک ہوتا ہے تاکہ اسٹاک ہوم میں 2 بڑوں اور 1 بچے کے لیے Airbnb کی اصلی رہائش تلاش کی جا سکے۔ تلاش کے معیار کو تبدیل کرنے کے لیے `user_inputs` فہرست میں ترمیم کریں۔


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# خلاصہ  
مبارک ہو! آپ نے کامیابی سے ایک AI ایجنٹ تیار کیا ہے جو Model Context Protocol (MCP) کے ذریعے حقیقی دنیا کی رہائش کی تلاش کے ساتھ یکجا ہوتا ہے:

## استعمال شدہ ٹیکنالوجیز:  
- Semantic Kernel - Azure OpenAI کے ساتھ ذہین ایجنٹس بنانے کے لیے  
- Microsoft Foundry - LLM صلاحیتوں اور چیٹ مکمل کرنے کے لیے  
- MCP (Model Context Protocol) - معیاری ٹول انضمام کے لیے  
- OpenBnB MCP سرور - حقیقی Airbnb تلاش کی فعالیت کے لیے  
- Node.js/NPX - بیرونی MCP سرور چلانے کے لیے  

## آپ نے کیا سیکھا:  
- MCP انضمام: Semantic Kernel ایجنٹس کو بیرونی MCP سرورز سے جوڑنا  
- حقیقی وقت میں ڈیٹا تک رسائی: حقیقی Airbnb پراپرٹیز کو لائیو APIs کے ذریعے تلاش کرنا  
- پروٹوکول مواصلات: ایجنٹ اور MCP سرور کے درمیان stdio کمیونیکیشن کا استعمال  
- فنکشن کی دریافت: MCP سرورز سے دستیاب فنکشنز خودکار طریقے سے دریافت کرنا  
- اسٹریمنگ جوابات: حقیقی وقت میں فنکشن کالز کو پکڑنا اور لاگ کرنا  
- HTML رینڈرنگ: طرزدار جدولوں اور انٹرایکٹو ڈسپلے کے ساتھ ایجنٹ کے جوابات کی فارمیٹنگ  

## اگلے اقدامات:  
- اضافی MCP سرورز کو انٹیگریٹ کرنا (موسم، پروازیں، ریستوران)  
- MCP اور A2A پروٹوکولز کو یکجا کرکے ایک ملٹی ایجنٹ سسٹم بنانا  
- اپنے ڈیٹا ذرائع کے لیے کسٹم MCP سرورز تیار کرنا  
- سیشنز کے دوران مستقل گفتگو کا میموری نافذ کرنا  
- MCP سرور آرکیسٹریشن کے ساتھ ایجنٹ کو Azure Functions پر تعینات کرنا  
- صارف کی توثیق اور بکنگ کی صلاحیتیں شامل کرنا


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**معذرت نامہ**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کے لیے کوشاں ہیں، براہ کرم اس بات کا خیال رکھیں کہ خودکار تراجم میں غلطیاں یا غلط فہمیاں ہو سکتی ہیں۔ اصل دستاویز اپنی مادری زبان میں معتبر ماخذ سمجھی جانی چاہیے۔ اہم معلومات کے لیے پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کی ذمہ داری ہم پر نہیں ہوگی۔
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
